# 中級：購入意向予測モデル構築（SparkML）

この章では、GOLD テーブルの属性・感情情報を使って、  
「購入意向（purchase_intent）」を予測する機械学習モデルを作ります。

手順の流れはこう：
1. 特徴量選定と前処理  
2. 訓練データ／検証データ分割  
3. SparkML パイプライン構築  
4. モデル学習／予測  
5. 評価（精度、F1スコアなど）  
6. モデルを使って新規予測も可能

シンプルなモデル（例えば LogisticRegression や RandomForest）から始めよう。

In [0]:
CATALOG = "hiroshi"
SCHEMA  = "survey_analysis_simple"

# UCオブジェクト作成
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE {SCHEMA}")

GOLD_TABLE_NAME = "gold_survey_responses_final"

## 必要ライブラリのインポートと GOLD データの取得

このセルでは SparkML に必要なクラスをインポートし、GOLD テーブルを読み込んで準備します。

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import functions as F

In [0]:
gold = spark.table(GOLD_TABLE_NAME)  # ← 実際の GOLD テーブル名
display(gold.limit(10))
gold.printSchema()

## 特徴量になるカテゴリ列を数値化（StringIndexer + OneHotEncoder）

このセルでは、カテゴリ変数（例: age_group, gender, region, positive_point_category, negative_feedback_category 等）を  
機械学習で使える形式（数値・ベクトル）に変換する処理を準備します。

In [0]:
# 例で使う列
categorical_cols = ["age_group", "gender", "region",
                    "positive_point_category", "negative_feedback_category"]

indexers = [StringIndexer(inputCol=c, outputCol=c + "_idx", handleInvalid="keep") for c in categorical_cols]
encoders = [OneHotEncoder(inputCol=c + "_idx", outputCol=c + "_vec") for c in categorical_cols]

# 数値列もそのまま使えるなら残す
numeric_cols = ["satisfaction_int"]  # 他にも追加できる

feature_cols = [c + "_vec" for c in categorical_cols] + numeric_cols

## 特徴ベクトルをまとめて "features" 列に

このセルでは、OneHotEncoder 出力 + 数値列を `VectorAssembler` で結合して、  
最終的な入力特徴量列 `features` を作ります。

In [0]:
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid="keep")

## ターゲットラベル列を数値化（StringIndexer）

このセルでは、目的変数 `purchase_intent` を数値ラベルに変換して `label` 列を作ります。

In [0]:
label_indexer = StringIndexer(inputCol="purchase_intent", outputCol="label", handleInvalid="keep")

## 訓練／検証データ分割とパイプライン構築

このセルでは、データを訓練セット／検証セットに分け、  
前処理 + 分類器を含む SparkML パイプラインを定義します。

In [0]:
train_df, test_df = gold.randomSplit([0.8, 0.2], seed=42)

# モデル選択（ここでは RandomForest を例に）
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=50)

pipeline = Pipeline(stages=indexers + encoders + [assembler, label_indexer, rf])

## モデル学習と予測

このセルでは、`pipeline.fit()` によってモデルを学習させ、  
検証データに対して予測を行います。

In [0]:
model = pipeline.fit(train_df)
pred_df = model.transform(test_df)
display(pred_df.select("purchase_intent", "label", "prediction", "probability").limit(20))

## モデル評価（精度、F1スコアなど）

このセルでは、`MulticlassClassificationEvaluator` を使って  
accuracy, f1, weightedPrecision, weightedRecall などを計算してモデル性能を評価します。

In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

for metric in ["accuracy", "f1", "weightedPrecision", "weightedRecall"]:
    score = evaluator.setMetricName(metric).evaluate(pred_df)
    print(f"{metric}: {score:.4f}")

## ✅ この章のまとめと振り返り

- 属性 + 感情カテゴリ情報を特徴量として、購入意向を予測するモデルを構築した  
- 評価指標を見ながら、どの特徴が効いているか仮説を立てる  
- 次のステップとしては、ハイパーパラメータチューニング（CrossValidator）、特徴選択、モデル比較（LogisticRegression / GBT / XGBoost 等）も試せる  
- また、学習済モデルを使って「この属性なら購入意向は何％か」予測出力・可視化するフェーズに進める  

このモデル構築が、実務で使える予測系分析の入口になるよ。  